In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords


In [2]:
#loading the data

data=pd.read_csv('data/imdb_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [3]:
data.head()


,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [5]:
data['sentiment'].value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [6]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words = []
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("movie is not good")

'movie is not good'

In [7]:
train_data = data[:int(0.7*len(data))]
test_and_dev_data = data[int(0.7*len(data)):]


dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())

print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     523 non-null    object
 1   sentiment  523 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.3+ KB
None



In [ ]:
print(test_data.info())


In [ ]:
print(dev_data.info())

In [ ]:
train_data['sentiment'].value_counts()

In [ ]:
test_data['sentiment'].value_counts()

In [ ]:
dev_data['sentiment'].value_counts()

In [8]:
X_train = train_data['review']
y_train = train_data['sentiment']
print(X_train)
print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
print(X_dev)
print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
print(X_test)
print(y_test)

0      A very, very, very slow-moving, aimless movie ...
1      Not sure who was more lost - the flat characte...
2      Attempting artiness with black & white and cle...
3           Very little music or anything to speak of.  
4      The best scene in the movie was when Gerardo i...
                             ...                        
518    The lines, the cuts, the audio, everything is ...
519             You can find better movies at youtube.  
520    Top line: Don't waste your time and money on t...
521    A Lassie movie which should have been "put to ...
522    That's how I'd describe this painfully dreary ...
Name: review, Length: 523, dtype: object
0      0
1      0
2      0
3      0
4      1
      ..
518    0
519    0
520    0
521    0
522    0
Name: sentiment, Length: 523, dtype: int64
523    So mediocre in every aspect that it just becom...
524    It isn't even an achievement as a "so-bad-it's...
525            It's an empty, hollow shell of a movie.  
526    Seriously, i

In [9]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.5583173996175909
Positive review class' prior probability-- 0.4416826003824092


In [21]:
def build_word_dict(data):
    word_freq = {}
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                else:
                    word_freq[word] += 1
  
    return word_freq

    
       

print("Total words--",len(build_word_dict(X_train)))


train_pos_reviews=train_data.loc[train_data.sentiment==1].review
pos_words_freq=build_word_dict(train_pos_reviews)
print("Positive words--",len(pos_words_freq))


train_neg_reviews=train_data.loc[train_data.sentiment==0].review
neg_words_freq=build_word_dict(train_neg_reviews)
print("Negative words--",len(neg_words_freq))

Total words-- 2482
Positive words-- 1267
Negative words-- 1748


In [22]:
neg_words_freq.pop('', None)

for word in neg_words_freq:
    print(word,"--",neg_words_freq[word])

a -- 93
drifting -- 1
slowmoving -- 1
man -- 4
aimless -- 1
very -- 21
movie -- 52
about -- 16
distressed -- 1
young -- 2
the -- 171
characters -- 10
of -- 82
out -- 14
was -- 61
flat -- 2
whom -- 2
not -- 30
or -- 18
walked -- 1
more -- 9
lost -- 1
nearly -- 1
half -- 5
audience -- 2
who -- 15
sure -- 3
almost -- 6
ridiculous -- 5
plot -- 16
white -- 1
with -- 29
attempting -- 1
black -- 4
as -- 26
clever -- 2
disappointed -- 4
nonexistent -- 1
artiness -- 1
camera -- 3
poor -- 6
lines -- 6
even -- 23
angles -- 1
and -- 89
acting -- 16
became -- 1
little -- 11
anything -- 4
music -- 4
to -- 70
speak -- 2
its -- 21
i -- 66
emptiness -- 1
because -- 12
art -- 3
works -- 2
if -- 15
charm -- 2
empty -- 1
meaning -- 2
it -- 71
rest -- 3
guess -- 4
lacks -- 4
hours -- 3
wasted -- 5
two -- 3
predictable -- 6
bit -- 4
budget -- 3
low -- 4
can -- 11
had -- 7
from -- 13
some -- 12
main -- 1
clearly -- 2
you -- 25
average -- 3
see -- 14
person -- 1
portrayal -- 1
are -- 22
nonsequel -- 1
talente

lucy -- 1
bell -- 1
sink -- 1
higher -- 1
slackers -- 1
actions -- 1
excuses -- 1
god -- 1
cover -- 3
tuneful -- 1
oldfashioned -- 1
pedestal -- 1
put -- 3
hayworth -- 1
rita -- 1
women -- 2
financial -- 1
example -- 3
hollywood -- 1
gain -- 1
exploit -- 1
seriously -- 1
fan -- 2
ticker -- 1
locations -- 1
couple -- 1
hummh -- 1
buildings -- 1
blew -- 1
chosen -- 1
we -- 3
propaganda -- 1
blatant -- 1
tortured -- 1
disgusting -- 1
american -- 1
clichs -- 2
elsewhere -- 1
americans -- 1
hatred -- 1
admitted -- 1
garbage -- 2
must -- 1
school -- 1
round -- 1
involved -- 3
pile -- 1
return -- 2
enter -- 1
values -- 2
admins -- 1
thunderbirds -- 1
place -- 3
sets -- 2
flakes -- 1
bordered -- 1
corn -- 1
style -- 2
backed -- 1
box -- 1
directorial -- 1
reason -- 3
repeats -- 1
stupidity -- 1
thrown -- 2
word -- 3
needlessly -- 1
slowmotion -- 1
continuity -- 2
vehicles -- 1
oh -- 2
star -- 1
proud -- 1
reviews -- 1
second -- 2
filmmaker -- 1
hide -- 1
head -- 1
sand -- 1
spent -- 2
nor -- 1

In [23]:
pos_words_freq.pop('', None)
len(pos_words_freq)
for word in pos_words_freq:
    print(word,"--",pos_words_freq[word])

movie -- 40
the -- 138
trying -- 1
is -- 66
running -- 1
song -- 2
his -- 10
to -- 56
through -- 3
that -- 30
scene -- 1
find -- 4
keeps -- 1
in -- 45
head -- 1
best -- 10
a -- 101
when -- 7
gerardo -- 1
was -- 37
saw -- 4
it -- 56
and -- 105
kids -- 2
effort -- 1
today -- 2
thought -- 3
good -- 16
for -- 24
messages -- 1
as -- 21
loved -- 4
of -- 78
casting -- 4
science -- 1
teacher -- 1
buffet -- 1
jimmy -- 2
owls -- 1
were -- 11
adorable -- 2
baby -- 1
those -- 3
florida -- 1
its -- 23
appealing -- 1
lot -- 1
made -- 7
look -- 7
at -- 13
very -- 14
showed -- 2
hilarious -- 3
so -- 17
songs -- 1
muppets -- 1
cool -- 4
almost -- 2
right -- 4
on -- 13
case -- 1
your -- 3
face -- 3
everything -- 2
this -- 66
delivers -- 2
sisters -- 2
two -- 1
i -- 64
review -- 1
greatest -- 2
single -- 3
long -- 3
ever -- 6
overdue -- 1
be -- 14
consider -- 2
tale -- 1
film -- 36
since -- 2
editing -- 2
up -- 4
or -- 6
against -- 1
ill -- 3
directing -- 2
postproduction -- 1
put -- 1
other -- 9
screenp

complex -- 1
paul -- 1
baileys -- 1
oscar -- 1
beautifully -- 1
colored -- 1
costumes -- 1
went -- 1
ishioka -- 1
dracula -- 1
eiko -- 1
crisp -- 1
notable -- 1
coppolas -- 1
design -- 1
small -- 1
annes -- 1
consolations -- 1
stunning -- 1
sublimely -- 1
moving -- 1
raw -- 1
plays -- 1
consequences -- 1
lie -- 1
monstrous -- 1
terror -- 1
overly -- 1
escalating -- 1
theatrical -- 1
stage -- 1
gripping -- 1
everyones -- 1
intelligent -- 1
without -- 1
players -- 1
main -- 1
mesmerising -- 1
broke -- 1
elses -- 1
hankies -- 1
males -- 1
fumbling -- 1
faces -- 1
wilkinson -- 1
judging -- 1
tom -- 1
females -- 1
alike -- 1
hands -- 1
fellowes -- 1
triumphed -- 1
julian -- 1
treasure -- 1
national -- 1
welldone -- 1
balance -- 1
phrase -- 1
owed -- 1
meaning -- 1
conflict -- 1
kristoffersen -- 1
difference -- 1
kris -- 1
documentaries -- 1
marriage -- 1
stars -- 1
punishment -- 1
park -- 1
using -- 1
shot -- 1
documentary -- 1
patent -- 1
southern -- 1
california -- 1
desert -- 1
watkins -

In [44]:
def conditional_probability(neg_words_freq, pos_words_freq):
    probability_of_word_given_negative = {}
    probability_of_word_given_positive = {}

    for word in neg_words_freq:
        probability_of_word_given_negative[word] = neg_words_freq[word] / len(train_pos_reviews)

    for word in pos_words_freq:
        probability_of_word_given_positive[word] = pos_words_freq[word] / len(train_neg_reviews)

    return probability_of_word_given_negative, probability_of_word_given_positive

probability_of_word_given_negative, probability_of_word_given_positive = conditional_probability(neg_words_freq, pos_words_freq)
#print(probability_of_word_given_negative)
